# Carregando bibliotecas necessárias

In [1]:
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
import pandas as pd

import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Configurando modelo de LLM a ser utilizado

In [2]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0.1,
    max_tokens=8000
)

# Dataset
Carregando os dados, e excluindo colunas que não serão utilizadas na proposta

In [4]:
livros = pd.read_csv("Books.csv", sep=",")
livros.drop('Image-URL-S', axis='columns', inplace=True)
livros.drop('Image-URL-M', axis='columns', inplace=True)
livros.drop('Image-URL-L', axis='columns', inplace=True)

C:\Users\paulo\AppData\Local\Temp\ipykernel_28168\1638965286.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  livros = pd.read_csv("Books.csv", sep=",")


In [5]:
# exibindo os dados de livros
livros.head()

ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  
0    Mark P. O. Morford                2002     Oxford University Press  
1  Richard Bruce Wright                2001       HarperFlamingo Canada  
2          Carlo D'Este                1991             HarperPerennial  
3      Gina Bari Kolata                1999        Farrar Straus Giroux  
4       E. J. W. Barber                1999  W. W. Norton &amp; Company

## Aplicando possíveis regras
A regra adotada é ter possíveis leitores maiores de dezoito anos e menores de 35 anos.

In [107]:
# filtrando usuario, total de 278858 para 86965 
usuarios = pd.read_csv("Users.csv", sep=",")
usuarios = usuarios[usuarios['Age'] >= 18]
usuarios = usuarios[usuarios['Age'] <= 35]
usuarios

User-ID                                Location   Age
1             2               stockton, california, usa  18.0
9            10              albacete, wisconsin, spain  26.0
12           13             barcelona, barcelona, spain  26.0
17           18  rio de janeiro, rio de janeiro, brazil  25.0
19           20            langhorne, pennsylvania, usa  19.0
...         ...                                     ...   ...
278843   278844                st. paul, minnesota, usa  28.0
278845   278846                toronto, ontario, canada  23.0
278848   278849             georgetown, ontario, canada  23.0
278850   278851                      dallas, texas, usa  33.0
278851   278852         brisbane, queensland, australia  32.0

[86965 rows x 3 columns]

In [108]:
# dividindo a base de usuários na metade
row_count = int(len(usuarios) / 2 )
row_count

43482

In [109]:
usuarios_supervisao = usuarios.iloc[0:row_count]

# definindo um limite de usuários para testar
limite_teste = 5
usuarios_recomendacao = usuarios.iloc[row_count: row_count + limite_teste]

In [110]:
usuarios_supervisao

User-ID                                Location   Age
1             2               stockton, california, usa  18.0
9            10              albacete, wisconsin, spain  26.0
12           13             barcelona, barcelona, spain  26.0
17           18  rio de janeiro, rio de janeiro, brazil  25.0
19           20            langhorne, pennsylvania, usa  19.0
...         ...                                     ...   ...
139935   139936          melbourne, victoria, australia  27.0
139941   139942                münchen, bayern, germany  29.0
139943   139944                     watauga, texas, usa  34.0
139944   139945                   gaa, alabama, vietnam  22.0
139946   139947              pasig, manila, philippines  21.0

[43482 rows x 3 columns]

In [111]:
usuarios_recomendacao

User-ID                                Location   Age
139948   139949      cambridge, england, united kingdom  30.0
139952   139953     vancouver, british columbia, canada  26.0
139953   139954  fingal bay, new south wales, australia  21.0
139955   139956                              olympia, ,  18.0
139956   139957                everett, washington, usa  22.0

In [115]:
# pegando livros com avaliação acima de 8 pontos, a ideia e recomendar somente livros muito bons
# somente livros bons e ajuda a diminuir o dataset de 433671 para 146151

rating = pd.read_csv("Ratings.csv", sep=",")
rating = rating[rating['Book-Rating'] >= 9]
rating

User-ID        ISBN  Book-Rating
9         276745   342310538           10
16        276747  0060517794            9
19        276747  0671537458            9
30        276760  8440682697           10
61        276772  3499230933           10
...          ...         ...          ...
1149744   276688  0836236688           10
1149745   276688  0892966548           10
1149775   276704  1563526298            9
1149777   276709  0515107662           10
1149778   276721  0590442449           10

[146151 rows x 3 columns]

In [116]:
# juntando a base de dados em uma única base, capaz de ter a informação de usuário, idade, livro e taxa de avaliação...
usuarios_supervisao_rating = pd.merge(usuarios_supervisao, rating, on='User-ID')
usuarios_supervisao_rating = pd.merge(usuarios_supervisao_rating, livros, on='ISBN')
usuarios_supervisao_rating

User-ID                                Location   Age        ISBN  \
0           51                 renton, washington, usa  34.0  0440225701   
1           56                  cheyenne, wyoming, usa  24.0  0679865691   
2           92  castellar del valles, barcelona, spain  20.0  8445071777   
3          183                  porto, porto, portugal  27.0  8401422825   
4          183                  porto, porto, portugal  27.0  847223973X   
...        ...                                     ...   ...         ...   
24254   139808                 syracuse, new york, usa  35.0  0472065211   
24255   139819         lincolnton, north carolina, usa  32.0  0312187459   
24256   139822                 clearwater, kansas, usa  20.0  0380719223   
24257   139900                everson, washington, usa  25.0  0553562614   
24258   139916                milano, lombardia, italy  35.0  0671727796   

       Book-Rating                                         Book-Title  \
0                9                                  The Street Lawyer   
1                9                         Haveli (Laurel Leaf Books)   
2               10  El Senor De Los Anillos: El Retorno Del Rey (T...   
3                9                                     Fahrenheit 451   
4                9                    Que Se Mueran Los Feos (Fabula)   
...            ...                                                ...   
24254            9  Simulacra and Simulation (The Body, In Theory:...   
24255            9                  The Memoirs of Cleopatra: A Novel   
24256            9                Mariel of Redwall (Redwall, Book 4)   
24257            9                                         Snow Crash   
24258           10                                   The Color Purple   

                  Book-Author Year-Of-Publication  \
0                JOHN GRISHAM                1999   
1      SUZANNE FISHER STAPLES                1995   
2            J. R. R. Tolkien                2001   
3                Ray Bradbury                1993   
4                  Boris Vian                2002   
...                       ...                 ...   
24254      Jean 0 Baudrillard                1995   
24255         Margaret George                1998   
24256           Brian Jacques                1999   
24257         Neal Stephenson                1993   
24258            Alice Walker                1990   

                              Publisher  
0                                  Dell  
1                           Laurel Leaf  
2                           Distribooks  
3      Plaza &amp; Janes Editores, S.A.  
4                              Tusquets  
...                                 ...  
24254                               UMP  
24255                St. Martin's Press  
24256                              Avon  
24257                            Bantam  
24258                            Pocket  

[24259 rows x 9 columns]

In [117]:
# criando o primeiro time de agentes
# agentes que irão entender o perfil do leitor de livros, agente que entenderá de livros e agente diretor de recomendação
perfil_leitor = Agent(
  role='perfil_leitor',
  goal='''A partir de dados limitados, você deve deduzir logicamente as conclusões sobre os leitores de livros.''',
  backstory='Você é um experiente psicólogo com décadas de experiência.',
  llm = llm,allow_delegation=False,verbose=True)

bibliotecario = Agent(
     role='bibliotecario',
     goal='''Indicar o livro adequado aos leitores''',
     backstory='Você tem um conhecimento excepcional em livros e pode dizer o quão valiosos eles são para um leitor.',
     llm = llm,allow_delegation=False,verbose=True)

Chief_Recommendation_Director = Agent(
     role="Chief Recomeendation Director",
     goal=dedent("""\Supervisione o trabalho feito pela sua equipe para garantir que seja o melhor
possível e alinhado com os objetivos de cada livro, revise, aprove,
faça perguntas esclarecedoras ou delegue trabalho de acompanhamento, se necessário, para tomar
decisões. Se necessário pesquisa a sinopse de cada livro, resumindo em no máximo 100 palavras"""),
     backstory=dedent("""\Você é o Diretor de uma grande empresa de venda de livros. Você está lançando uma campanha de anúncios personalizada,
tentando garantir que sua equipe esteja criando o melhor
conteúdo possível para os leitores."""),
     llm = llm,tools=[],allow_delegation=False, verbose=True)

# Segundo time de agente para fazer a recomendação de livros com base no perfil do leitor
campaign_agent = Agent(
     role="campaign_agent",
     goal=dedent("""\Desenvolver conteúdo atraente e inovador
para campanhas publicitárias, com foco em cópias de anúncios específicas para os leitores de livros."""),
     backstory=dedent("""\Como um Criador de Conteúdo Criativo em uma agência de marketing digital
de primeira linha, você se destaca na criação de anúncios
que ressoam com clientes em potencial.
Sua especialidade está em transformar estratégias de marketing
em histórias envolventes que capturam
a atenção e inspiram ações de compra."""),
     llm = llm,allow_delegation=False, verbose=True)

In [118]:
# primeira tarefa para o agente
def get_ad_campaign_task(agent, descricao_leitore, lista_livros_usuario_rating):
  return Task(description=dedent(f"""\
    Você está criando uma campanha de marketing direcionada, adaptada ao que sabemos sobre nossos leitores.

    Para cada leitor, temos que escolher exatamente três livros com maior Book-Rating para promover na próxima campanha.
    Certifique-se de que a seleção seja a melhor possível e alinhada com o perfil dos leitores,
    revise, aprove, faça perguntas esclarecedoras ou delegue trabalho de acompanhamento, se
    necessário para tomar decisões. Ao delegar trabalho, envie o rascunho completo
    como parte das informações.
    Esta é a lista de todos os leitores que já leram os livros, com a média de idade dos leitores e a taxa de avaliação para cada livro: {lista_livros_usuario_rating}.
    Isto é tudo o que sabemos até agora sobre o perfil do novo leitor que deveremos fazer a indicação dos livros: {descricao_leitore}.
    
    Para iniciar esta campanha, precisaremos primeiro construir uma compreensão do nosso leitor.
    Assim que tivermos um perfil sobre os leitores que já leram os livros e sobre o novo leitor,
    temos que selecionar exatamente três livros que tenham a maior chance de serem comprados pelo novo leitor.
    
    Sua resposta final DEVE ser exatamente 3 livros da lista, cada um com uma breve sinopse
    por que este livro corresponde ao novo leitor. Deve ser formatado como este exemplo:
     :
     :
     :
    """),
    agent=agent,expected_output='Uma versão finalizada e refinada da campanha de marketing em formato markdown'
  )

# segunda tarefa para o agente
def get_ad_campaign_written_task(agent, selection):
    return Task(description=dedent(f"""\

    Você está criando uma campanha de marketing direcionada, adaptada ao que sabemos sobre nossos leitores.

    Para cada leitor, escolhemos três livros para promover na próxima campanha.
    Esta seleção é adaptada especificamente para o leitor: {selection},
    
    Para encerrar esta campanha com sucesso, precisaremos de uma mensagem promocional anunciando esses livros para o leitor com a intenção final de que ele compre de nós.
    Esta mensagem deve ter cerca de 20 palavras, para que possa ser facilmente integrada à carta completa. Por exemplo:
    Interessado em ler sobre ciência de dados, compre este livor da Universidade de Harvard.
    Leve sua leitura para o próximo nível com a ajuda deste livro.
    
    Você precisa revisar, aprovar e delegar o trabalho de acompanhamento, se necessário, para ter a mensagem promocional completa. Ao delegar o trabalho, envie o rascunho completo
    como parte das informações.
    
    Sua resposta final DEVE incluir os 3 livros da lista, cada um com uma curta mensagem promocional.
    
    """),
    agent=agent,expected_output='Uma versão finalizada e refinada da campanha de marketing em formato markdown'
  )

In [119]:
# iniciando a execução dos agentes

df_output_list = [] 
# colunas da base
# Location 	Age 	ISBN 	Book-Rating
for index, row in usuarios_recomendacao.iterrows():
  print('############################################## '+ str(index))
  descricao_leitor = f'''
  Leitores que moram na localização: {row['Location']}.
  Com média de idade de {row['Age']} anos.
  '''
  print(descricao_leitor)
  

# definindo a tarefa 1 para selecionar os três melhores livros
  task1 = get_ad_campaign_task(Chief_Recommendation_Director, descricao_leitor, usuarios_supervisao_rating)
  # Iniciando o time de agentes
  targetting_crew = Crew(
    agents=[perfil_leitor, bibliotecario ,Chief_Recommendation_Director ],
    tasks=[task1],
    verbose=True, 
  process=Process.sequential 
  )
  targetting_result = targetting_crew.kickoff()
  
 
# definindo a tarefa 2 para a geração da campanha de recomendação com base no perfil de possíveis leitores
  task2 = get_ad_campaign_written_task(Chief_Recommendation_Director ,targetting_result)
  copywriting_crew = Crew(
    agents=[campaign_agent,Chief_Recommendation_Director ],
    tasks=[task2],
    verbose=True, 
  process=Process.sequential
  )
  copywriting_result = copywriting_crew.kickoff()

  # criando a saída do resultado
  df_output_list.append({'leitor':descricao_leitor,
                         'livros_direcionados':targetting_result,
                         'promo_msg':copywriting_result,
                        })


############################################## 139948

  Leitores que moram na localização: cambridge, england, united kingdom.
  Com média de idade de 30.0 anos.
  


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a8d1e506-729f-47fa-a10b-05d9c33543ac                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b76f4a2d-bf67-4d5b-b4e3-6f6099334ecf
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b76f4a2d-bf67-4d5b-b4e3-6f6099334ecf
       Status: Executing Task...
    └── 🤖 Agent: Chief Recomeendation Director
            Status: In Progress

# Agent: Chief Recomeendation Director
## Task:     Você está criando uma campanha de marketing direcionada, adaptada ao que sabemos sobre nossos leitores.

    Para cada leitor, temos que escolher exatamente três livros com maior Book-Rating para promover na próxima campanha.
    Certifique-se de que a seleção seja a melhor possível e alinhada com o perfil dos leitores,
    revise, aprove, faça perguntas esclarecedoras ou delegue trabalho de acompanhamento, se
    necessário para tomar decisões. Ao delegar trabalho, envie o rascunho completo
    como parte das informações.
    Esta é a lista de todos os leitores que já leram os livros, com a média de idade dos leitores e a taxa de avaliação para cada livro:        User-ID                                Location   Age        ISBN  \
0           51                 renton, washington, usa  34.0  0440225701   
1           56                  cheyenne, wyoming, usa  24.0  0679865691   
2           92  castellar del valles, barcelona, spain

...



# Agent: Chief Recomeendation Director
## Final Answer: 
# Campanha de Marketing Personalizada

Olá, leitor de Everett, Washington, EUA! Temos três livros incríveis selecionados especialmente para você. Confira:

## 1. "O Apanhador no Campo de Centeio" por J.D. Salinger
> Um romance de amadurecimento que segue a história de Holden Caulfield, um adolescente desiludido navegando pelas complexidades da adolescência. Com temas de alienação, identidade e rebeldia, este livro ressoa com jovens leitores que estão em busca de seu lugar no mundo.

## 2. "O Sol é para Todos" por Harper Lee
> Um clássico ambientado no sul dos Estados Unidos na década de 1930. Através dos olhos de Scout Finch, uma jovem que cresce em uma sociedade racialmente dividida, o livro explora temas de racismo, injustiça e o poder da empatia. Esta história atemporal ensina valiosas lições sobre compaixão e defender o que é certo.

## 3. "O Grande Gatsby" por F. Scott Fitzgerald
> Uma obra-prima literária que captura a es

🚀 Crew: crew
└── 📋 Task: a73ce113-c166-465b-adaa-495787c7201d
       Status: Executing Task...
    └── 🤖 Agent: Chief Recomeendation Director
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: a73ce113-c166-465b-adaa-495787c7201d
       Assigned to: Chief Recomeendation Director
       Status: ✅ Completed
    └── 🤖 Agent: Chief Recomeendation Director
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a73ce113-c166-465b-adaa-495787c7201d                                                                     │
│  Agent: Chief Recomeendation Director                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1e186fa0-108d-4277-9c38-6277a8b9bf4d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [121]:
# Collect results in dataframe
df_output = pd.DataFrame(df_output_list)
df_output

leitor  \
0  \n  Leitores que moram na localização: cambrid...   
1  \n  Leitores que moram na localização: vancouv...   
2  \n  Leitores que moram na localização: fingal ...   
3  \n  Leitores que moram na localização: olympia...   
4  \n  Leitores que moram na localização: everett...   

                                 livros_direcionados  \
0  Based on the information provided, the goal is...   
1  Based on the information provided, the new rea...   
2  Based on the information provided, the new rea...   
3  Based on the information provided, the new rea...   
4  1. Book Title: The Catcher in the Rye\n   Book...   

                                           promo_msg  
0  # Marketing Campaign\n\n## Book 1: "Pride and ...  
1  # Campanha de Marketing Personalizada\n\nOlá, ...  
2  # Marketing Campaign\n\n## Book 1: "To Kill a ...  
3  # Book Promotions\n\n## 1. "The Catcher in the...  
4  # Campanha de Marketing Personalizada\n\nOlá, ...

In [124]:
# imprimindo a primeira mensagem criada para avaliar
df_output["promo_msg"][0].raw

'# Marketing Campaign\n\n## Book 1: "Pride and Prejudice" by Jane Austen\n- Synopsis: Explore love, marriage, and social class in 19th-century England. Join Elizabeth Bennet on her journey to find love in unexpected places.\n- Promotional Message: "Discover the timeless tale of love and societal expectations in \'Pride and Prejudice\' by Jane Austen. Get lost in the world of 19th-century England."\n\n## Book 2: "To Kill a Mockingbird" by Harper Lee\n- Synopsis: Dive into the racially divided town of Maycomb, Alabama, during the 1930s. Experience Scout Finch\'s perspective on racial injustice, morality, and the loss of innocence.\n- Promotional Message: "Uncover the powerful story of justice and morality in \'To Kill a Mockingbird\' by Harper Lee. Witness the impact of racial injustice through Scout Finch\'s eyes."\n\n## Book 3: "The Great Gatsby" by F. Scott Fitzgerald\n- Synopsis: Immerse yourself in the decadence and disillusionment of the Roaring Twenties. Follow Jay Gatsby\'s pursu

In [134]:
# exibindo o resultado da execução no formato de marcadores
from IPython.display import Markdown

perfil = df_output['leitor'][0]
msg= f"Para {perfil} Os livros indicados são: "
print(msg)

Markdown(df_output["promo_msg"][0].raw)

Para 
  Leitores que moram na localização: cambridge, england, united kingdom.
  Com média de idade de 30.0 anos.
   Os livros indicados são: 


# Marketing Campaign

## Book 1: "Pride and Prejudice" by Jane Austen
- Synopsis: Explore love, marriage, and social class in 19th-century England. Join Elizabeth Bennet on her journey to find love in unexpected places.
- Promotional Message: "Discover the timeless tale of love and societal expectations in 'Pride and Prejudice' by Jane Austen. Get lost in the world of 19th-century England."

## Book 2: "To Kill a Mockingbird" by Harper Lee
- Synopsis: Dive into the racially divided town of Maycomb, Alabama, during the 1930s. Experience Scout Finch's perspective on racial injustice, morality, and the loss of innocence.
- Promotional Message: "Uncover the powerful story of justice and morality in 'To Kill a Mockingbird' by Harper Lee. Witness the impact of racial injustice through Scout Finch's eyes."

## Book 3: "The Great Gatsby" by F. Scott Fitzgerald
- Synopsis: Immerse yourself in the decadence and disillusionment of the Roaring Twenties. Follow Jay Gatsby's pursuit of the American Dream and the emptiness of wealth and materialism.
- Promotional Message: "Step into the glamorous world of the Roaring Twenties with 'The Great Gatsby' by F. Scott Fitzgerald. Experience the allure and tragedy of the American Dream."

Don't miss out on these literary masterpieces! Grab your copies today and embark on unforgettable journeys through time and society.

Please review and provide any feedback or further instructions.